In [1]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 67.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=dfb31a8c323573b02393737c10e1399b85cbf9611cf06d62d493cb0fcaafa796
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import pyspark as ps
from pyspark.ml.feature import StopWordsRemover
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
twitter_file="/content/gdrive/MyDrive/Colab Notebooks/sentiment140.csv"

In [5]:
conf = ps.SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.driver.memory', '16g')])
#sc.stop()
sc = ps.SparkContext(conf=conf)

In [6]:
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
sql_context = SQLContext(sc)




/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [7]:
twitterDf = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(twitter_file)

twitterDf.show()

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  0|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  0|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  0|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  0|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  0|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  0|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [8]:
from pyspark.sql.functions import regexp_replace

twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "@[A-Za-z0-9]+", ""))
twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "(?:\@|http?\://|https?\://|www)\S+", ""))
#tw = tw.withColumn("Tweet", regexp_replace("Tweet", "#[A-Za-z0-9]+", ""))
twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "'", ""))
twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "[^A-Za-z0-9]+", " "))
twitterDf = twitterDf.withColumn("_c5", f.lower(f.col("_c5")) )

In [9]:
twitterDf.tail(5)

[Row(_c0='4', _c1='2193601966', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='AmandaMarie1028', _c5='just woke up having no school is the best feeling ever '),
 Row(_c0='4', _c1='2193601969', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='TheWDBoards', _c5='thewdb com very cool to hear old walt interviews '),
 Row(_c0='4', _c1='2193601991', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='bpbabe', _c5='are you ready for your mojo makeover ask me for details '),
 Row(_c0='4', _c1='2193602064', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='tinydiamondz', _c5='happy 38th birthday to my boo of alll time tupac amaru shakur '),
 Row(_c0='4', _c1='2193602129', _c2='Tue Jun 16 08:40:50 PDT 2009', _c3='NO_QUERY', _c4='RyanTrevMorris', _c5='happy charitytuesday ')]

In [10]:
stopwordList = nltk.corpus.stopwords.words('english')

In [11]:
from pyspark.sql.types import DoubleType

twitterDf = twitterDf.withColumn("_c0", twitterDf["_c0"].cast(DoubleType()))

In [12]:

trainDF, testDF = twitterDf.randomSplit([0.80, 0.20], seed=2000)

In [13]:
testDF.show(5,truncate=False)

+---+----------+----------------------------+--------+---------------+----------------------------------------------------------------------------------------+
|_c0|_c1       |_c2                         |_c3     |_c4            |_c5                                                                                     |
+---+----------+----------------------------+--------+---------------+----------------------------------------------------------------------------------------+
|0.0|1467812723|Mon Apr 06 22:20:19 PDT 2009|NO_QUERY|TLeC           | i couldnt bear to watch it and i thought the ua loss was embarrassing                  |
|0.0|1467812771|Mon Apr 06 22:20:19 PDT 2009|NO_QUERY|robrobbierobert| it it counts idk why i did either you never talk to me anymore                         |
|0.0|1467812799|Mon Apr 06 22:20:20 PDT 2009|NO_QUERY|HairByJess     | i wish i got to watch it with you i miss you and how was the premiere                  |
|0.0|1467814119|Mon Apr 06 22:20:40 PDT 

In [14]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [15]:
tokenizer = Tokenizer(inputCol="_c5", outputCol="words")
stopwordremover=StopWordsRemover(inputCol="words", outputCol="TweetWithStopwordsfree")
hashtf = HashingTF(numFeatures=2**16, inputCol="TweetWithStopwordsfree", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
#label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer,stopwordremover, hashtf, idf])

pipelineFit = pipeline.fit(twitterDf)


In [16]:
trainDF = pipelineFit.transform(trainDF)
testDF = pipelineFit.transform(testDF)
trainDF.show(5)

+---+----------+--------------------+--------+---------------+--------------------+--------------------+----------------------+--------------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|               words|TweetWithStopwordsfree|                  tf|            features|
+---+----------+--------------------+--------+---------------+--------------------+--------------------+----------------------+--------------------+--------------------+
|0.0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_| a thats a bummer...|[, a, thats, a, b...|  [, thats, bummer,...|(65536,[10345,216...|(65536,[10345,216...|
|0.0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|[is, upset, that,...|  [upset, cant, upd...|(65536,[16064,193...|(65536,[16064,193...|
|0.0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus| i dived many tim...|[, i, dived, many...|  [, dived, many, t...|(65536,[2548,2888...|(6

In [17]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
trainDF=trainDF.selectExpr("_c0 as label", "_c5 as text","words","TweetWithStopwordsfree", "tf","features")
lrModel = lr.fit(trainDF)
testDF=testDF.selectExpr("_c0 as label", "_c5 as text","words","TweetWithStopwordsfree", "tf","features")
predictions = lrModel.transform(testDF)

In [18]:
predictions.show(5)

+-----+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|                text|               words|TweetWithStopwordsfree|                  tf|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0| i couldnt bear t...|[, i, couldnt, be...|  [, couldnt, bear,...|(65536,[513,8997,...|(65536,[513,8997,...|[9.30412012010187...|[0.92843932358713...|       0.0|
|  0.0| it it counts idk...|[, it, it, counts...|  [, counts, idk, e...|(65536,[6589,1383...|(65536,[6589,1383...|[9.16107302381454...|[0.90692659025276...|       0.0|
|  0.0| i wish i got to ...|[, i, wish, i, go...|  [, wish, got, wat...|(65536,[20719,292...|(65536,[20719,292...|[9.62126269400567...|[0.96072507588899...|    

In [19]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator


evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.7703392688829602

In [21]:
tweets1 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/tuesdaycategoryPredictedDataset/categoryPredictedDataset0.csv')
tweets2 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/tuesdaycategoryPredictedDataset/categoryPredictedDataset1.csv')
tweets3 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/tuesdaycategoryPredictedDataset/categoryPredictedDataset2.csv')
tweets4 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/tuesdaycategoryPredictedDataset/categoryPredictedDataset4.csv')

In [22]:
tweets1.show()
tweets2.show()
tweets3.show()
tweets4.show()

+--------------------+-------------+
|                 _c0|          _c1|
+--------------------+-------------+
|check out r kive ...|         tech|
|craig and gizmo r...|        sport|
|you have nothing ...|        sport|
|if your group cha...|entertainment|
|if i ever watch t...|         tech|
|fortnite playing ...|     business|
|i wanna play a ga...|        sport|
|        36 years old|entertainment|
|we cannoh replace...|    political|
|all the stochasti...|    political|
|such class and gr...|entertainment|
|britain and the u...|        sport|
|day 85 run 43 380...|        sport|
|hand selected lik...|        sport|
|we had productive...|     business|
|neil lennon was a...|        sport|
|     nanana cover by|entertainment|
|this is their pla...|    political|
|quality swag is a...|        sport|
|trump s next coup...|    political|
+--------------------+-------------+
only showing top 20 rows

+--------------------+-------------+
|                 _c0|          _c1|
+-----------

In [23]:
import functools
  
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [24]:
tweets = unionAll([tweets2, tweets1])
print(tweets.count())

187058


In [25]:
tweets = unionAll([tweets, tweets3])
print(tweets.count())

315168


In [26]:
tweets = unionAll([tweets, tweets4])
print(tweets.count())

364322


In [27]:
tweets.show()

+--------------------+-------------+
|                 _c0|          _c1|
+--------------------+-------------+
|i m going to be i...|        sport|
|biden s controver...|entertainment|
|in case no one ha...|        sport|
|jam tangan casio ...|     business|
|ap b brbad ho ge phr|     business|
|norris who still ...|        sport|
|putin controls biden|     business|
|he shrinks in big...|         tech|
|i dunno what ever...|        sport|
|omg tears in my eyes|entertainment|
|        done join dc|    political|
|             explain|        sport|
|please support me...|        sport|
|ashes 2021 joe ro...|        sport|
|all my vids are o...|         tech|
|croc s world 60 o...|         tech|
|the unvaccinated ...|    political|
|feel good story o...|entertainment|
|our nation is bei...|    political|
|          still here|        sport|
+--------------------+-------------+
only showing top 20 rows



In [28]:
tweets = tweets.selectExpr("_c0 as _c5","_c1 as catlabel")

In [29]:
tweets.show()

+--------------------+-------------+
|                 _c5|     catlabel|
+--------------------+-------------+
|i m going to be i...|        sport|
|biden s controver...|entertainment|
|in case no one ha...|        sport|
|jam tangan casio ...|     business|
|ap b brbad ho ge phr|     business|
|norris who still ...|        sport|
|putin controls biden|     business|
|he shrinks in big...|         tech|
|i dunno what ever...|        sport|
|omg tears in my eyes|entertainment|
|        done join dc|    political|
|             explain|        sport|
|please support me...|        sport|
|ashes 2021 joe ro...|        sport|
|all my vids are o...|         tech|
|croc s world 60 o...|         tech|
|the unvaccinated ...|    political|
|feel good story o...|entertainment|
|our nation is bei...|    political|
|          still here|        sport|
+--------------------+-------------+
only showing top 20 rows



In [30]:
sentimantpredictions = pipelineFit.transform(tweets)

In [31]:
sentimantpredictions = lrModel.transform(sentimantpredictions)

In [32]:
sentimantpredictions.show()

+--------------------+-------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                 _c5|     catlabel|               words|TweetWithStopwordsfree|                  tf|            features|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|i m going to be i...|        sport|[i, m, going, to,...|  [m, going, ev, sa...|(65536,[7777,9739...|(65536,[7777,9739...|[5.68577655166498...|[0.00925271760610...|       4.0|
|biden s controver...|entertainment|[biden, s, contro...|  [biden, controver...|(65536,[997,11233...|(65536,[997,11233...|[7.10226147721198...|[0.13704720485558...|       4.0|
|in case no one ha...|        sport|[in, case, no, on...|  [case, one, told,...|(65536,[4919,7713...|(65536,[4919,7713..

In [33]:
sentimantpredictions=sentimantpredictions.drop("words","TweetWithStopwordsfree","tf","features","rawPrediction","probability")

In [34]:
sentimantpredictions.show()

+--------------------+-------------+----------+
|                 _c5|     catlabel|prediction|
+--------------------+-------------+----------+
|i m going to be i...|        sport|       4.0|
|biden s controver...|entertainment|       4.0|
|in case no one ha...|        sport|       4.0|
|jam tangan casio ...|     business|       4.0|
|ap b brbad ho ge phr|     business|       0.0|
|norris who still ...|        sport|       4.0|
|putin controls biden|     business|       4.0|
|he shrinks in big...|         tech|       4.0|
|i dunno what ever...|        sport|       4.0|
|omg tears in my eyes|entertainment|       0.0|
|        done join dc|    political|       4.0|
|             explain|        sport|       4.0|
|please support me...|        sport|       4.0|
|ashes 2021 joe ro...|        sport|       4.0|
|all my vids are o...|         tech|       0.0|
|croc s world 60 o...|         tech|       0.0|
|the unvaccinated ...|    political|       4.0|
|feel good story o...|entertainment|    

In [35]:
sentimantpredictions.write.csv("/content/gdrive/MyDrive/Colab Notebooks/tuesdaydatasetWithPredictedCategoryAndSentimentDecSix")